In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load the dataset first

In [ ]:
cars_df = pd.read_csv("/kaggle/input/craigslist-carstrucks-data/vehicles.csv")

In [ ]:
cars_df.head()

In [ ]:
cars_df.shape

Find out manufacturer's counts

In [ ]:
cars_manufacture = cars_df["manufacturer"].value_counts()
print(cars_manufacture)

In [ ]:
import seaborn as sns

**Plot the top 10 used cars by manufacturers**

In [ ]:
top_seller=cars_df["manufacturer"].value_counts()[0:10]
sns.barplot(y=top_seller.index, x=top_seller.values);

Find cars that won't much sold as used cars (Bottom Sellers)

In [ ]:
bottom_seller=cars_df["manufacturer"].value_counts()[-10:]
sns.barplot(y=bottom_seller.index, x=bottom_seller.values, orient="h");

#   **Interesting.. Let's find out why Ford is sold so much as compared to other cars**

Is it price the culprit??

Find average pricing of bottom cars

In [ ]:
car_price_mean=cars_df.groupby(["manufacturer"])["price"].mean()
y_val=[]
for b,c in bottom_seller.items():
    y_val.append(car_price_mean[b])
g=sns.barplot(x=bottom_seller.index, y=y_val)
g.set_xticklabels(g.get_xticklabels(), rotation=45)

Find out the average price of top seller cars

In [ ]:
y_val=[]
for b,c in top_seller.items():
    y_val.append(car_price_mean[b])
g=sns.barplot(x=top_seller.index, y=y_val)
g.set_xticklabels(g.get_xticklabels(), rotation=45)

There are many top-seller and bottom-seller cars comes in range of lower than 25000. Then, **PRICE is not a differential factor.**

Is it condition of vehicle thats a factor that buyer is opting

In [ ]:
bottom_seller=cars_df["manufacturer"].value_counts()[-10:]
filter_cars_df = cars_df[cars_df["manufacturer"].isin(bottom_seller.index)]

In [ ]:
(filter_cars_df
.groupby("manufacturer")["condition"]
.value_counts(normalize=True)
.mul(100)
.rename('percent')
.reset_index()
.pipe((sns.catplot,'data'), x="percent",y='manufacturer',hue="condition",kind='bar'))

In [ ]:
top_seller=cars_df["manufacturer"].value_counts()[0:10]
filter_cars_df = cars_df[cars_df["manufacturer"].isin(top_seller.index)]
(filter_cars_df
.groupby("manufacturer")["condition"]
.value_counts(normalize=True)
.mul(100)
.rename('percent')
.reset_index()
.pipe((sns.catplot,'data'), x="percent",y='manufacturer',hue="condition",kind='bar'))

Hmm.. Looks like top seller has approx 50% excellent condition vehicles. But some of bottom seller cars also has more than 50% excellent conditions of vehicle like aston, harley-davidson etc. If you check for their average price, it is in range of 20K-25K.

Also, if you consider only the top-seller, all have same conditions, then why Ford is able to make the difference.

* **Is it type of vehicle??**

Till now, we were trying to get the plot the attributes independently. Consider now with more attributes together

Let's consider only excellent, like new and good vehicle and price will below 25K

Top selling vehicle types

In [ ]:
top_types = cars_df["type"].value_counts()[0:5]

In [ ]:
filter_cars_df=cars_df[cars_df["condition"].isin(["excellent", "good", "like new"])]
filter_cars_df=filter_cars_df[filter_cars_df["price"]<25000]
filter_cars_df = filter_cars_df[filter_cars_df["type"].isin(top_types.index)]

In [ ]:
top_seller=cars_df["manufacturer"].value_counts()[0:7]
filter_top_df = filter_cars_df[filter_cars_df["manufacturer"].isin(top_seller.index)]

In [ ]:
(filter_top_df
.groupby("manufacturer")["type"]
.value_counts(normalize=False)
.rename('sold units')
.reset_index()
.pipe((sns.catplot,'data'), x="sold units",y='manufacturer',hue="type",kind='bar'))

Sedan is commonly famous across most of the top sellers. So, it is not differential factor.

# Pickup and Truck are the 2 major difference that creates the difference among the top sellers.

In [ ]:
filter_bottom_df = filter_cars_df[filter_cars_df["manufacturer"].isin(bottom_seller.index)]

In [ ]:
(filter_bottom_df
.groupby("manufacturer")["type"]
.value_counts()
.rename('sold units')
.reset_index()
.pipe((sns.catplot,'data'), x="sold units",y='manufacturer',hue="type",kind='bar'))

Output

# Final Verdict - Buyers who are more interested towards truck and pickup type vehicles, they prefer to go for "Ford" with excellent and good conditions vehicle and with range of 25K price